In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import numpy as np
import copy
import json
import pickle

In [2]:
def missing_values_preprocess(dt_fr,flag):
    dt_fr.drop(columns = ['Loan_ID'],inplace =True)
    if flag == 0:
        num_feat = [var for var in dt_fr.columns if  dt_fr[var].isnull().mean() > 0 and dt_fr[var].dtypes == 'float']
        feat_1 = [var for var in dt_fr.columns if  dt_fr[var].isnull().mean() > 0 and dt_fr[var].dtypes == 'object']
        feat_2 = [var for var in dt_fr.columns if var not in feat_1 and dt_fr[var].dtypes == 'object']
        feat_1.append('Credit_History')
        emp_lst = []
        for var in feat_1:
            emp_lst.append(dt_fr[var].mode())
            #print(emp_lst)
        dt_fr1 = dt_fr.fillna({'Gender':emp_lst[0][0],'Married':emp_lst[1][0],'Dependents':emp_lst[2][0],'Self_Employed':emp_lst[3][0]})
        t1 = dt_fr['Credit_History'].mode()
        dt_fr2 = dt_fr1.fillna({'LoanAmount':dt_fr1['LoanAmount'].mean(), 'Loan_Amount_Term':dt_fr1['Loan_Amount_Term'].mean(), 'Credit_History':emp_lst[4][0]})
        if dt_fr2.isnull().sum().sum() == 0 :
            return (dt_fr2,feat_1,feat_2)
    else:
        
        dt_fr.dropna(inplace =True)
        feat_1 = [var for var in dt_fr.columns if  dt_fr[var].dtypes == 'object']
        #feat_2 = [var for var in dt_fr.columns if var not in feat_1 and dt_fr[var].dtypes == 'object']
        dt_fr.reset_index(drop = True,inplace = True)
        if dt_fr.isnull().sum().sum() == 0 :
            return (dt_fr,feat_1)       

In [3]:
le = LabelEncoder()
def label_encoding(df_1,feat_lst):
    Cat_labl_dct = {}
    for var in feat_lst:
        df_1[var] = le.fit_transform(df_1[var])
        mapping = dict(zip(le.classes_, map(int, le.transform(le.classes_))))
        Cat_labl_dct[var] = mapping
        json_object = json.dumps(Cat_labl_dct, indent=4)
        with open("sample.json", "w") as outfile:
            json.dump(Cat_labl_dct, outfile)
    return df_1

In [4]:
ohe = OneHotEncoder()
def O_H_Encd(ohe_lst,l2,df_2):
    X_tmp = ohe.fit_transform(df_2[ohe_lst]).toarray()
    X_tmp = X_tmp[:,(0,2,4,5,7)]
    df_ohe = pd.DataFrame(X_tmp,columns = ['Gender','Married','Self_Employed_1','Self_Employed_2','Property_Area'])
    df_tmp = df_2[l2]
    df_ohe_fn = pd.concat([ df_ohe,df_tmp],axis = 1)
    return df_ohe_fn


In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import confusion_matrix
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment('Loan_Prediction_1')
def Model(X_train,X_Val,Y_train,Y_Val ):
    with mlflow.start_run(run_name = 'P_Decision_tree') as parent:
        Parameters_P = {'max_depth' : 8 , 'min_samples_leaf' : 5}
        mlflow.log_params(Parameters_P)
        dt=DecisionTreeClassifier(max_depth = 5,min_samples_leaf = 3)
        dt.fit(X_train,Y_train)
        Prediction_Tr = dt.predict(X_train)
        Prediction_Val = dt.predict(X_Val)
        Prediction_Tst = dt.predict(df_tst_rtrn.iloc[:])
        Score_Tr = accuracy_score(Y_train,Prediction_Tr)
        Score_Val = accuracy_score(Y_Val,Prediction_Val)
        filename = 'finalized_model.sav'
        pickle.dump(dt, open(filename, 'wb'))
        print(f'The accuracy for Training dataset is {Score_Tr}')
        print(f'The accuracy for Validation dataset is {Score_Val}')
        print(f'The f1 score for Training dataset is {f1_score(Y_train,Prediction_Tr)}')
        print(f'The f1 score for Validation dataset is {f1_score(Y_Val,Prediction_Val)}')
        metrics_dict = {'Acc_tr':Score_Tr,'Acc_val':Score_Val,'f1_score_training':f1_score(Y_train,Prediction_Tr),'f1_score_Validation':f1_score(Y_Val,Prediction_Val)}
        mlflow.log_metrics(metrics_dict)
        mlflow.sklearn.log_model(dt,'SK_models')
        
        return (Prediction_Tr,Prediction_Val,Prediction_Tst)
       

In [6]:
def Preparing_fnl_Df(Prediction_Tr,Prediction_Val,flag):
    if flag == 'train':
        tmp = pd.DataFrame(data = Prediction_Tr,columns = ['Predicted'])
        tmp1 = pd.DataFrame(data = Prediction_Val,columns = ['Predicted'])
        tmp = pd.concat([tmp,tmp1],axis = 0)
        df_fn_train['Prediction'] = tmp.values
        df_fn_train.loc[df_fn_train['Prediction'] == 1,'Prediction'] = 'y'
        df_fn_train.loc[df_fn_train['Prediction'] == 0,'Prediction'] = 'N'
        return  df_fn_train
    if flag == 'test':
        df_fn_test['Prediction'] = Prediction_Tr
        df_fn_test.loc[df_fn_test['Prediction'] == 1,'Prediction'] = 'y'
        df_fn_test.loc[df_fn_test['Prediction'] == 0,'Prediction'] = 'N'
        return  df_fn_test
        

In [7]:
df_tr = pd.read_csv('/home/anuj/Documents/ANUJ_Project/RADHA_My_frst_project_preplaced/Loan_pred/Training Dataset.csv')
df_tst = pd.read_csv('/home/anuj/Documents/ANUJ_Project/RADHA_My_frst_project_preplaced/Loan_pred/Test Dataset.csv')


In [8]:
df_tr_rtrn,cat_feat,cat_feat_1 = missing_values_preprocess(df_tr,0)
df_tst_rtrn,cat_feat_tst = missing_values_preprocess(df_tst,1)
df_fn_test= copy.deepcopy(df_tst_rtrn)
df_fn_train = copy.deepcopy(df_tr_rtrn)


In [9]:
df_tr_rtrn = label_encoding(df_tr_rtrn,cat_feat) 
df_tr_rtrn = label_encoding(df_tr_rtrn,cat_feat_1)
df_tst_rtrn = label_encoding(df_tst_rtrn,cat_feat_tst)
df_tr_rtrn['Loan_Status'] = df_tr['Loan_Status'] 
df_tr_rtrn['Loan_Status'] = le.fit_transform(df_tr['Loan_Status'])


In [10]:
# lst_2 = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount','Loan_Amount_Term', 'Credit_History', 'Property_Area','Education','Loan_Status']
# lst_1 = ['Gender','Married','Self_Employed','Property_Area']
# lst_3 = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount','Loan_Amount_Term', 'Credit_History', 'Property_Area','Education']
# df_tr_final = O_H_Encd(lst_1,lst_2,df_tr_rtrn)
# df_tst_final = O_H_Encd(lst_1,lst_3,df_tst_rtrn)


In [11]:
#X_tr,X_V,Y_tr,Y_V = train_test_split(df_tr_final.iloc[:,0:-1],df_tr_final.iloc[:,-1],test_size=0.2,random_state=42,shuffle=False)
X_tr,X_V,Y_tr,Y_V = train_test_split(df_tr_rtrn.iloc[:,0:-1],df_tr_rtrn.iloc[:,-1],test_size=0.2,random_state=42,shuffle=False)
print(X_tr.shape)
Prediction_Tr,Prediction_Val,Prediction_Tst = Model(X_tr,X_V,Y_tr,Y_V )

(491, 11)
The accuracy for Training dataset is 0.8289205702647657
The accuracy for Validation dataset is 0.7804878048780488
The f1 score for Training dataset is 0.8855585831062671
The f1 score for Validation dataset is 0.8524590163934426


In [12]:
df_train_set_otpt = Preparing_fnl_Df(Prediction_Tr,Prediction_Val,'train')
df_test_set_otpt = Preparing_fnl_Df(Prediction_Tst,Prediction_Val,'test')